In [31]:
import time
import numpy as np

### Hash function

In [19]:
def hash_fun(string):
    m = 32582657 
    chunck_s = [string[0:8],string[8:16],string[16:24],string[24:32]] #Since each line is 32 longs, we split it into four parts of 8
    r=[2,17,4,32]
    h=0
    for i in range(len(chunck_s)):
        x = encoding(chunck_s[i]) #invoke the encoding function to convert each str into ASCII char
        h += (int(x)*r[i])%m #compute the hashing
    binary = bin(h)[2:] #make binary the result
    str_binary = str(binary)
    return (str_binary,h) #return the obtained vector of str

In [20]:
def encoding(string):
    s=0
    for char in string:
        s+=ord(char) #convert each str into ASCII char
    return s

### Hyperloglog

In [42]:
def leftmost_zeros(bin_dig):
    i=1
    while bin_dig[-i]==0 and i<len(bin_dig):
        i+=1
    return i

In [43]:
def hll(file):
    n = 12
    m = 2**n
    length_file = 100000 #139000000 file intero
    #initialization of the buckets
    buckets = [0 for x in range (m)]
    f = open(file, "r") #open the file
    #for each rows in the file
    for row in range(35,length_file):
        #read one line at time
        line = f.readline(row) 
        #compute the hash function on it
        digit = hash_fun(line) 
        #add zeros to obtain 32 bit
        digit_32 = '0'*4+digit[0]+'0'*(28-len(digit[0])) 
        
        #add items to the buckets
        j = 1+int(digit_32[:n-1],2)
        w=digit_32[n:]
        buckets[j]=max(buckets[j],leftmost_zeros(w))
           
    #computing cardinality 
    alfa = 0.7213/(1+1.079/(2**n))
    Z=0
    for i in buckets:
        Z+=2**(-buckets[i]-1)
    Z=Z**(-1)
    E = alfa*(m)**2*Z #cardinality
    return (E)

### Read the file and save the length

In [6]:
file = "small_hash.txt" #smaller file
#file = "hash.txt" #smaller file
def read_file(file_red):
    with open(file_red) as f:
        lines = f.read().splitlines()
    n_lines = len(lines)
    return (n_lines)

length_file= read_file(file) #save the dimension of the file
print(length_file)
# 139000000 file intero

100000


### Compute cardinality

In [44]:
start = time.time()
card = hll(file)#,length_file)
end = time.time()

print('Execution time is:',round(end-start,3), ' seconds' )
print('Cardinality: ', card)

Execution time is: 0.259  seconds
Cardinality:  5908.0546426109995


### Compute error

In [ ]:
#print('Loglog: ', np.log(np.log(card)))
real_len = 125000001 # obtained with set()
error = real_len/(abs(real_len-card)/card)*100
print('Error: ',error)